### Sedaro API "Hello World" Example

This notebook exercises the most basic example of:
- Querying the API
- Creating a new block
- Running a scenario
- Viewing results

#### Important: Read Before Running

This notebook makes changes to agent and scenario branches indicated in the settings section. Ensure any changes to the target branches are saved prior to running this code. Sedaro recommends committing current work and creating new branches in the target repositories to avoid loss of work.

This notebook also requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
    "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.

In [ ]:
import json

import matplotlib.pyplot as plt
from sedaro import SedaroApiClient, SedaroSimulationResult
from sedaro_base_client.apis.tags import jobs_api

In [ ]:
# Settings
with open('./secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']

with open('./config.json', 'r') as file:
    config = json.load(file)

# Obtain these IDs from the branch list within each repository and add to config.json
AGENT_TEMPLATE_BRANCH_ID = config['AGENT_TEMPLATE_BRANCH_ID']    # ID of the vehicle template branch
SCENARIO_BRANCH_ID = config['SCENARIO_BRANCH_ID']                # ID of the scenario template branch
HOST = config['HOST']                                            # Sedaro instance URL

In [ ]:
def get_by_name(block_class, name):
    '''Search for a block with a particular name.'''
    instances = block_class.get_all()
    results = [entry for entry in instances if entry.name == name]
    if len(results) > 0:
        return results[0]
    else:
        return None

#### Query Simulation Elements
The API allows for queries of elements in the simulation, commonly referred to as "blocks". Here we query every block
of the `SubSystem` type that groups spacecraft components for analytics. For each subsystem, we print its name and the
name of every `Component` under it. Any element of the simulation can be queried in this manner.

In [ ]:
with SedaroApiClient(api_key=API_KEY, host=HOST) as sedaro_client:
    branch_client = sedaro_client.get_branch(AGENT_TEMPLATE_BRANCH_ID)
    for subsystem in branch_client.Subsystem.get_all():
        print(subsystem.name)
        for component in subsystem.components:
            print("    -", component.name)


#### Create Simulation Elements

Here a new `AngularVelocitySensor` is instantiated using the `branch_client.<BLOCK_NAME>.create` method. See the `sedaro` package documentation [here](https://github.com/sedaro/sedaro-python/tree/main/sedaro) for a listing of available
blocks.

In [ ]:
with SedaroApiClient(api_key=API_KEY, host=HOST) as sedaro_client:
    branch_client = sedaro_client.get_branch(AGENT_TEMPLATE_BRANCH_ID)

    gnc_subsystem = get_by_name(branch_client.Subsystem, 'GNC')
    sensor_name = 'Test Sensor'
    sensor = get_by_name(branch_client.AngularVelocitySensor, sensor_name)
    if sensor is None:
        sensor = branch_client.AngularVelocitySensor.create(
            name=sensor_name,
            oneSigmaPerAxisError={'deg/s': 0.01},
            subsystem=gnc_subsystem.id
        )


#### Simulate & Plot Results

Now we will use the API to start the simulation and plot some of the results. The results of this run will also appear in the web interface as usual.

In [ ]:
with SedaroApiClient(api_key=API_KEY, host=HOST) as sedaro_client:
    # Instantiate API and scenario objects
    sim = sedaro_client.get_sim_client(SCENARIO_BRANCH_ID)

    # Start scenario and wait for it to finish, then get data
    sim.start()
    results = SedaroSimulationResult.poll_scenario_latest(API_KEY, SCENARIO_BRANCH_ID, host=HOST)

    # Summarize results
    results.summarize()

In [ ]:
if results.success:
    for agent_name in results.templated_agents:
        agent_results = results.agent(agent_name)
        if sensor.id in agent_results:
            # Select sensor results and show summary
            sensor_result = agent_results.block(sensor.id)
            sensor_result.summarize()

            # Plot Data
            sensor_result.measurement.rpm.plot(show=False, marker='.', markersize=2, linewidth=0)
            plt.ylabel('Angular Velocity (rpm)')
            plt.legend(['x', 'y', 'z'])
            plt.show()